In [3]:
import os
import requests
import json
from datetime import datetime
import mysql.connector
from dotenv import load_dotenv
import logging
import time


In [3]:
load_dotenv()

True

In [7]:
def fetch_mandi_data():
    """Fetch complete mandi data from the API using pagination"""
    try:
        api_key = os.getenv('MANDI_API_KEY')
        if not api_key:
            raise ValueError("MANDI_API_KEY not found in environment variables")
        

        base_url = "https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24"
        limit = 5000
        offset = 0
        all_records = []

        while True:
            url = f"{base_url}?api-key={api_key}&format=json&limit={limit}&offset={offset}&filters%5BArrival_Date%5D=2025-05-23"
            response = requests.get(url)
            response.raise_for_status()

            data = response.json()
            records = data.get('records', [])

            if not records:
                break  # No more data to fetch

            all_records.extend(records)
            logging.info(f"Fetched {len(records)} records at offset {offset}")

            offset += limit  # Move to next page
            time.sleep(1)

        return all_records

    except Exception as e:
        logging.error(f"Error fetching mandi data: {str(e)}")
        raise


In [8]:
r = fetch_mandi_data()

In [9]:
len(r)

18943

In [13]:
import pandas as pd
df = pd.DataFrame(r)

In [14]:
df.head(5)

,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min_Price,Max_Price,Modal_Price,Commodity_Code
0,Madhya Pradesh,Shivpuri,Barad,Groundnut,Big (With Shell),FAQ,17/05/2025,4000,5030,4600,10
1,Madhya Pradesh,Neemuch,Neemuch,Coriander(Leaves),Coriander,FAQ,17/05/2025,6400,7600,7600,43
2,Madhya Pradesh,Damoh,Damoh,Bengal Gram(Gram)(Whole),Gram,FAQ,17/05/2025,5155,5855,5670,6
3,Madhya Pradesh,Shajapur,Shujalpur,Bengal Gram(Gram)(Whole),Gram,FAQ,17/05/2025,4920,9300,5681,6
4,Madhya Pradesh,Vidisha,Sironj,Bengal Gram(Gram)(Whole),Gram,FAQ,17/05/2025,5300,5630,5565,6


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18494 entries, 0 to 18493
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   State           18494 non-null  object
 1   District        18494 non-null  object
 2   Market          18494 non-null  object
 3   Commodity       18494 non-null  object
 4   Variety         18494 non-null  object
 5   Grade           18494 non-null  object
 6   Arrival_Date    18494 non-null  object
 7   Min_Price       18494 non-null  object
 8   Max_Price       18494 non-null  object
 9   Modal_Price     18494 non-null  object
 10  Commodity_Code  18494 non-null  object
dtypes: object(11)
memory usage: 1.6+ MB
